In [1]:
from databroker.v2 import temp
from bluesky import RunEngine
from secop_ophyd.secop_ophyd.SECoPDevices import SECoP_Node_Device, SECoPReadableDevice, SECoP_CMD_Device
import asyncio
import random



# Create a run engine and a temporary file backed database. Send all the documents from the RE into that database
RE = RunEngine({})
db = temp()
RE.subscribe(db.v1.insert)

from bluesky.plans import scan
from ophyd.sim import det, motor

#Example of adding metadata to RE environment

investigation_id = "kmnk2n34"

RE.md["investigation_id"] = investigation_id

robo_dev = SECoP_Node_Device.create_external_loop('localhost','10770',RE.loop)

# now do something
RE(scan([det],motor,1,2,10),reason="for a demo")






sample_changer.HZB ready


('1edd6549-0de4-4588-a4f6-477f8fce0f6d',)

In [2]:
await robo_dev.read_configuration()

storage:SECoPReadableDevice= robo_dev.storage

reading = await storage.value.read()

loadshort:SECoP_CMD_Device = storage.load_short_dev


reading = await storage.value.read()
print(reading)


loadshort:SECoP_CMD_Device = storage.load_short_dev

for samplepos in range(1,13):
    await loadshort.samplepos_arg.set(samplepos)
    await loadshort.substance_arg.set(random.randint(0,6))

    await loadshort.load_short_x.execute()

    await storage.wait_for_IDLE()
    print(samplepos)
        
        







{'sample_changer-HZB-storage-value': {'value': [], 'timestamp': 1692874978.0038111}}
1
2
3
4
5
6
7
8
9
10
11
12


In [10]:
await robo_dev.storage.read()
#await robo_dev.storage.read_configuration()

{'sample_changer-HZB-storage-value': {'value': ["{'sample_name': 'NugatRittersport', 'substance': 'Nugat', 'substance_code': '4', 'sample_pos': 1, 'manufacturer': 'Rittersport', 'sample_id': '4b84b2b6-1d53-4eef-8b00-5050e414a505', 'color': 'dark blue', 'mass': 0.0167}",
   "{'sample_name': 'Nuss-SplitterRittersport', 'substance': 'Nuss-Splitter', 'substance_code': '3', 'sample_pos': 2, 'manufacturer': 'Rittersport', 'sample_id': '4f0ed37e-e8d4-40c4-a68a-23adf2e70fdc', 'color': 'green', 'mass': 0.0167}",
   "{'sample_name': 'NugatRittersport', 'substance': 'Nugat', 'substance_code': '4', 'sample_pos': 3, 'manufacturer': 'Rittersport', 'sample_id': '5c166e53-b464-4f28-8ab6-33fe8b7cdb2e', 'color': 'dark blue', 'mass': 0.0167}",
   "{'sample_name': 'MarzipanRittersport', 'substance': 'Marzipan', 'substance_code': '5', 'sample_pos': 4, 'manufacturer': 'Rittersport', 'sample_id': 'd02fe8ca-30d0-4304-9414-4a6f01ae6961', 'color': 'red', 'mass': 0.0167}",
   "{'sample_name': 'MarzipanRitterspor